In [5]:
import pickle
import numpy as np
import os

def load_cifar_batch(filename):
    with open(filename, 'rb') as f:
        batch = pickle.load(f, encoding='bytes')
        data = batch[b'data']
        labels = batch[b'labels']
        data = data.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
        return data, labels

def load_cifar10(root_dir):
    x_train = []
    y_train = []

    # Load training batches
    for i in range(1, 6):
        data_batch, labels_batch = load_cifar_batch(os.path.join(root_dir, f'data_batch_{i}'))
        x_train.append(data_batch)
        y_train.append(labels_batch)

    x_train = np.concatenate(x_train)
    y_train = np.concatenate(y_train)

    # Load test batch
    x_test, y_test = load_cifar_batch(os.path.join(root_dir, 'test_batch'))
    y_test = np.array(y_test)

    return (x_train, y_train), (x_test, y_test)

# Set the path to your extracted CIFAR-10 folder
cifar10_dir = r'C:\Users\PENSION\.keras\datasets\cifar-10-python\cifar-10-batches-py'
(x_train, y_train), (x_test, y_test) = load_cifar10(cifar10_dir)

print("Train shape:", x_train.shape)
print("Test shape:", x_test.shape)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\PENSION\\.keras\\datasets\\cifar-10-python\\cifar-10-batches-py\\data_batch_1'

In [3]:
from tensorflow.keras.datasets import cifar10

# This should now load successfully!
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


Exception: URL fetch failure on https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load CIFAR-10 dataset
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0

# Build the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10)  # 10 output classes for CIFAR-10
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc:.2f}")

# Save the model
model.save("my_model.h5")
print("✅ Model saved as my_model.h5")


Exception: URL fetch failure on https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)